In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import os
from datetime import datetime
from pprint import pprint

import database
import helper
import utils

In [ ]:
LOG = utils.CustomLogger("CustomLogger", log_level= "info", display_loglevel= False, display_datetime= False)
PICKLE_LIB = utils.PickleLib(data_path="./data", logger= LOG)
JIRA = "Jira" 

# Data Search

In [ ]:
client = database.connect()
db = client.JiraRepos
collection = db[JIRA]

#### Summary

In [ ]:
def getInRange():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$lt": [{"$strLenCP": "$fields.summary"}, 70]},
                        {"$gt": [{"$strLenCP": "$fields.summary"}, 39]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 5
            }
        }
    ])
    return results

def getShorter():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$lt": [{"$strLenCP": "$fields.summary"}, 39]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 5
            }
        }
    ])
    return results

def getLonger():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$gt": [{"$strLenCP": "$fields.summary"}, 70]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 10
            }
        }
    ])
    return results

In [ ]:
summary = []

# results = getInRange()
# results = getShorter()
results = getLonger()

for document in results:
    summary.append(document)
    print("Id: " + str(document['id']) + " Length: " + str(len(document['fields']['summary'])) + ": " + document['fields']['summary'])

#### Description Structure

In [ ]:
def findKeysByCode(data, target_code):
    matching_keys = []
    for key, value in data.items():
        if 'code' in value and value['code'] == target_code:
            matching_keys.append(key)
    return matching_keys

In [ ]:
with open('issueTypeMapping.json') as f:
    mappedIssueTypes = json.load(f)

In [ ]:
mappedTypes = findKeysByCode(mappedIssueTypes[JIRA], 'Bug Report')
mappedTypes

In [ ]:
tickets = []
results = collection.aggregate([
    {
        "$match": {
            "fields.description": {
                "$exists": True, 
                "$type": "string",
                "$regex": "Steps to"
            },
            "fields.issuetype.name": {
                "$in": mappedTypes
            }
        }
    },
    {
        "$sample": {
            "size": 30
        }
    }
])

for document in results:
    tickets.append(document)
    print("Id", document['id'])


# Ticket Creation

In [ ]:
### Globals ###
LOG.reset()
FOLDERNAME = "examples"

In [ ]:
evo_df = PICKLE_LIB.pickle_load("./jiraEvolutions/load_evolution_dataframe(jiras=[_"+JIRA+"_])", 'gzip')
# evo_df = PICKLE_LIB.pickle_load("./jiraEvolutions/load_evolution_dataframe(sample_data_n=10000)", 'gzip')

In [ ]:
# tickets = evo_df[evo_df["jira"]== JIRA]
# bugReports = tickets[tickets["data_to"].isin(mappedTypes)]
# sample_id = bugReports['issue_id'].values[3]
sample_id = tickets[12]['id']
samples = evo_df[evo_df["issue_id"] == sample_id]
sample = samples[samples["jira"]== JIRA]
sample

In [ ]:
evolutionStep = 0
ticket = helper.createTicket(sample, evolutionStep)
ticket

In [ ]:
print(ticket['Description'].values[0])

# Ticket PreProcessing

In [ ]:
def preprocessTickets(ticket):
    
    ### Convert CreatedDate and ResolvedDate to datetime
    c_date = ticket['CreatedDate'].values[0]
    c_dt_obj = datetime.strptime(c_date, '%Y-%m-%dT%H:%M:%S.%f%z')
    ticket['CreatedDate'] = c_dt_obj.strftime('%Y-%m-%d %H:%M:%S')
    r_date = ticket['ResolvedDate'].values[0]
    r_dt_obj = datetime.strptime(r_date, '%Y-%m-%dT%H:%M:%S.%f%z')
    ticket['ResolvedDate'] = r_dt_obj.strftime('%Y-%m-%d %H:%M:%S')

    ### Convert IssueId to Int
    id = ticket['IssueId'].values[0]
    ticket['IssueId'] = int(id)

preprocessTickets(ticket)

In [ ]:
helper.saveTicket(FOLDERNAME, ticket, evolutionStep, JIRA, sample_id)

# Annotate Ticket

#### Update

In [ ]:
def annotateTicket(ticket, annotation, reason):
    try:
        ticket['ViolationActual'] = annotation
        ticket['ViolationReason'] = reason
        print("Annotation successful.")
    except:
        print("Annotation failed.")

annotateTicket(ticket, "TRUE", """1. Resolution has to be set from 'None' to 'Low Priority'.
                           2. Status has to be set from 'Open' to 'Resolved'.""")

In [ ]:
if os.path.isfile("data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv"):
    dataset = pd.read_csv("./data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv")
    print("The dataset already exists.")
else:
    dataset = pd.DataFrame(columns=['Jira', 'IssueId', 'EvoId', 'Summary', 'Description', 'VersionsAffected', 'IssueType', 'Project', 'Components', 'CreatedDate', 'ResolvedDate', 'Status', 'Priority', 'Creator', 'Reporter', 'Resolution', 'IssueLinks', 'Labels','VersionsFixed', 'Assignee', 'TimeSpent', 'Comments', 'ViolationActual', 'ViolationReason', 'FieldCount', 'Fields'])
    print("The dataset was created successfully.")

dataset = pd.concat([dataset, ticket], ignore_index=True)

dataset.to_csv("data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv", index=False)
print("The ticket was inserted into the dataset successfully!")


In [ ]:
dataset = pd.read_csv('data/' + FOLDERNAME + '/' + FOLDERNAME + 'Dataset.csv')
dataset

#### Bug Report Structure

In [ ]:
def annotateTicket(ticket, annotation, reason):
    try:
        ticket['SmellActual'] = annotation
        ticket['SmellReason'] = reason
        print("Annotation successful.")
    except:
        print("Annotation failed.")

#annotateTicket(ticket, "3", """No previous structure. The model has to generate a new structure from scratch.""")
annotateTicket(ticket, "2", """The structure contains some parts, but is not "complete".""")
#annotateTicket(ticket, "1", """The structure contains all important parts or was only been slightly modified.""")

In [ ]:
if os.path.isfile("data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv"):
    dataset = pd.read_csv("./data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv")
    print("The dataset already exists.")
else:
    dataset = pd.DataFrame(columns=['Jira', 'IssueId', 'EvoId', 'Summary', 'Description', 'VersionsAffected', 'IssueType', 'Project', 'Components', 'CreatedDate', 'ResolvedDate', 'Status', 'Priority', 'Creator', 'Reporter', 'Resolution', 'IssueLinks', 'Labels','VersionsFixed', 'Assignee', 'TimeSpent', 'Comments', 'ViolationActual', 'ViolationReason', 'FieldCount', 'Fields'])
    print("The dataset was created successfully.")

dataset = pd.concat([dataset, ticket], ignore_index=True)

dataset.to_csv("data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv", index=False)
print("The ticket was inserted into the dataset successfully!")

In [ ]:
dataset = pd.read_csv('data/' + FOLDERNAME + '/' + FOLDERNAME + 'Dataset.csv')
dataset